# Project Minerva
## Notebook: The Global Risks Interconnections Map 2017
### https://libre-ai.github.io/minerva/

In [318]:
import math
from collections import Counter, defaultdict
import igraph as ig
import plotly
from plotly.graph_objs import *
import json

In [319]:
class Node:
    name = ""
    group = ""
    
    def __init__(self, name, group):
        self.name = name
        self.group = group
        
    def to_map(self):
        return {"name": self.name, "group": self.group}
        
    def to_json_string(self):
        return "{\"name\":\"%s\", \"group\":%d}"%(self.name, self.group)

In [320]:
class Link:
    source = 0
    target = 0
    value = 1
    
    def __init__(self, source, target, value):
        self.source = source
        self.target = target
        self.value = value
        
    def to_map(self):
        return {"source": self.source, "target": self.target, "value": self.value}
        
    def to_json_string(self):
        return "{\"source\":\"%s\", \"target\":%d, \"value\":%d}"%(self.source, self.target, self.value)

In [321]:
indexed_risk_tables = eval("[(0, 'risk_001_asset_bubbles_in_a_major_economy'), (1, 'risk_002_deflation_in_a_major_economy'), (2, 'risk_003_failure_of_a_major_financial_mechanism_or_institution'), (3, 'risk_004_failure_shortfall_of_critical_infrastructure'), (4, 'risk_005_fiscal_crises_in_key_economies'), (5, 'risk_006_high_structural_unemployment_or_underemployment'), (6, 'risk_007_illicit_trade'), (7, 'risk_008_severe_energy_price_shock'), (8, 'risk_009_unmanageable_inflation'), (9, 'risk_010_extreme_weather_events'), (10, 'risk_011_failure_of_climate_change_mitigation_and_adaptation'), (11, 'risk_012_major_biodiversity_loss_and_ecosystem_collapse'), (12, 'risk_013_man_made_environmental_damage_and_disasters'), (13, 'risk_014_failure_of_national_governance'), (14, 'risk_015_failure_of_regional_or_global_governance'), (15, 'risk_016_interstate_conflict_with_regional_consequences'), (16, 'risk_017_large_scale_terrorist_attacks'), (17, 'risk_018_state_collapse_or_crisis'), (18, 'risk_019_weapons_of_mass_destruction'), (19, 'risk_020_failure_of_urban_planning'), (20, 'risk_021_food_crises'), (21, 'risk_022_large_scale_involuntary_migration'), (22, 'risk_023_profound_social_instability'), (23, 'risk_024_rapid_and_massive_spread_of_infectious_diseases'), (24, 'risk_025_water_crises'), (25, 'risk_026_adverse_consequences_of_technological_advances'), (26, 'risk_027_breakdown_of_critical_information_infrastructure_and_networks_and_cyberattacks'), (27, 'risk_028_massive_incident_of_data_fraud_and_theft')]")

In [322]:
#indexed_risk_tables

In [323]:
risk_groups = {
    0:"Economic Risks",
    1:"Environmental Risks",
    2:"Geopolitical Risks",
    3:"Societal Risks",
    4:"Technological Risks"
    }

In [324]:
nodes = [
    Node("Asset bubbles in a major economy", 0),
    Node("Deflation in a major economy", 0),
    Node("Failure of a major financial mechanism or institution", 0),
    Node("Failure/shortfall of critical infrastructure", 0),
    Node("Fiscal crises in key economies", 0),
    Node("High structural unemployment or underemployment", 0),
    Node("Illicit trade", 0),
    Node("Severe energy price shock", 0),
    Node("Unmanageable inflation", 0),
    #---
    Node("Extreme weather events", 1),
    Node("Failure of climate-change mitigation and adaptation", 1),
    Node("Major biodiversity loss and ecosystem collapse", 1),
    Node("Man made environmental damage and disasters", 1),
    #---
    Node("Failure of national governance", 2),
    Node("Failure of regional or global governance", 2),
    Node("Interstate conflict with regional consequences", 2),
    Node("Large scale terrorist attacks", 2),
    Node("State collapse or crisis", 2),
    Node("Weapons of mass destruction", 2),
    #---
    Node("Failure of urban planning", 3),
    Node("Food crises", 3),
    Node("Large scale involuntary migration", 3),
    Node("Profound social instability", 3),
    Node("Rapid and massive spread of infectious diseases", 3),
    Node("Water crises", 3),
    #---
    Node("Adverse consequences of technological advances", 4),
    Node("Breakdown of critical information infrastructure and networks and cyberattacks", 4),
    Node("Massive incident of data fraud and theft", 4)
]

In [325]:
indexed_nodes = zip(range(len(nodes)), nodes)

#### The file `pairwise_weights.txt` can be found at [https://github.com/libre-ai/minerva/tree/master/data]

In [326]:
link_weights = eval(open("./pairwise_weights.txt").readlines()[0].strip())

In [327]:
def topn_links(nodes, link_weights, topn=5, remove_same_group_links=False):
    topn_nodes = set()
    node2links = defaultdict(Counter)
    for ((node_idx_a, node_idx_b), weight) in link_weights:
        node_a = nodes[node_idx_a]
        node_b = nodes[node_idx_b]
        if remove_same_group_links and node_a.group == node_b.group:
            continue
        #keep track for each node in the link of associated weight
        node2links[node_idx_a][(node_idx_a, node_idx_b)] = weight
        node2links[node_idx_b][(node_idx_a, node_idx_b)] = weight
    for node_idx in node2links:
        links_of_node = node2links[node_idx]
        topn_nodes.update(set(links_of_node.most_common(topn)))
    return list(topn_nodes)
    

In [328]:
links_base = map(lambda x: Link(x[0][0], 
                           x[0][1],
                           int(round(x[1] * 100))
                          ),
            topn_links(nodes, link_weights, topn=4, remove_same_group_links=True)
           )

links = [l for l in links_base]

In [329]:
graph_source = repr(
        {"nodes": map(lambda x: x.to_map(), nodes),
         "links": map(lambda x: x.to_map(), links)
        }
    ).replace("'", "\"")


In [331]:
print(graph_source)

In [ ]:
# with open("global_risks_source.json", "w") as fout:
#     fout.write(graph_source)

---

### The output `global_risks_source.json` can be found at:

#### https://github.com/libre-ai/minerva/tree/master/data

#### and the source of the resulting visualization can be found at:

#### https://github.com/libre-ai/minerva/blob/gh-pages/index.html , 

#### which can be seen at: https://libre-ai.github.io/minerva/

---


---